# РТ 8: Кластеризация

Изучение методов кластеризации данных

**Содержание:**

- Метод k-средних (k-means)
- Иерархическая кластеризация
- Применение на реальных данных

ТО ЧТО ВНИЗУ ОБЯЗАТЕЛЬНО ЗАПУСТИТЬ ЕСЛИ НЕ В КОЛАБЕ ОТКРЫТО


In [ ]:
import sys
!{sys.executable} -m pip install -q numpy pandas matplotlib scikit-learn scipy seaborn


## Часть 1: Метод k-средних (k-means)

Кластеризация — разбиение множества объектов на подмножества (кластеры) так, чтобы объекты в одном кластере были схожи, а в разных — различались.

### Пример - Генерация данных и применение k-means


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

# Генерируем 2D-набор данных с 4 кластерами
X, y_true = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

print("Генерация данных с 4 кластерами")
print(f"Размер данных: {X.shape}")
print(f"Истинные метки (для визуализации): {np.unique(y_true)}")

# Визуализация исходных данных
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(X[:, 0], X[:, 1], s=50, alpha=0.6, edgecolors='black')
plt.title('Исходные данные (без меток)')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.grid(True, alpha=0.3)

# Применяем k-means с k=4
kmeans = KMeans(n_clusters=4, random_state=0, n_init=10)
y_kmeans = kmeans.fit_predict(X)

# Визуализация результатов кластеризации
plt.subplot(1, 2, 2)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis', 
            alpha=0.6, edgecolors='black')
# Отображаем центроиды
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.8, 
            marker='X', edgecolors='black', linewidths=2, label='Центроиды')
plt.title('Результат k-means кластеризации (k=4)')
plt.xlabel('Признак 1')
plt.ylabel('Признак 2')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nЦентры кластеров:")
for i, center in enumerate(centers):
    print(f"  Кластер {i}: {center}")
    
print(f"\nИнерция (сумма квадратов расстояний до центроидов): {kmeans.inertia_:.2f}")


### Пример - K-means на наборе цифр

Применение k-means к набору рукописных цифр для их группировки


In [ ]:
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# Загрузка набора данных цифр
digits = load_digits()
print("Набор данных цифр:")
print(f"  Размер данных: {digits.data.shape}")
print(f"  Количество изображений: {len(digits.images)}")
print(f"  Размер изображения: {digits.images[0].shape}")

# Применяем k-means с 10 кластерами (по числу цифр 0-9)
kmeans_digits = KMeans(n_clusters=10, random_state=0, n_init=10)
clusters = kmeans_digits.fit_predict(digits.data)

print(f"\nК ластеризация завершена!")
print(f"Количество кластеров: 10")

# Визуализация центроидов кластеров
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    # Центроид кластера - это средний образ цифры
    ax.imshow(kmeans_digits.cluster_centers_[i].reshape(8, 8), cmap='gray')
    ax.set_title(f'Кластер {i}')
    ax.axis('off')
plt.suptitle('Центроиды кластеров (средние образы цифр)')
plt.tight_layout()
plt.show()

# Анализ: какие цифры попали в каждый кластер
print("\nАнализ кластеров:")
for i in range(10):
    mask = clusters == i
    digits_in_cluster = digits.target[mask]
    unique, counts = np.unique(digits_in_cluster, return_counts=True)
    print(f"Кластер {i}: преобладающая цифра = {unique[np.argmax(counts)]}, "
          f"распределение: {dict(zip(unique, counts))}")


## Задание 1 - Кластеризация заданного массива данных

**Задача:** Провести кластерный анализ данных методом k-средних. Поэкспериментировать с количеством кластеров.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Заданный массив данных
X = np.array([[5, 3],
              [10, 15],
              [15, 12],
              [24, 10],
              [30, 45],
              [85, 70],
              [71, 80],
              [60, 78],
              [55, 52],
              [80, 91]])

print("Заданный массив данных:")
print(X)
print(f"\nРазмер данных: {X.shape}")

# Эксперименты с разным количеством кластеров
k_values = [2, 3, 4, 5]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for idx, k in enumerate(k_values):
    # Применяем k-means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    y_pred = kmeans.fit_predict(X)
    centers = kmeans.cluster_centers_
    
    # Визуализация
    ax = axes[idx // 2, idx % 2]
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y_pred, s=200, cmap='viridis',
                        alpha=0.6, edgecolors='black', linewidths=2)
    ax.scatter(centers[:, 0], centers[:, 1], c='red', s=300, alpha=0.9,
              marker='X', edgecolors='black', linewidths=3, label='Центроиды')
    
    # Подписываем точки
    for i, point in enumerate(X):
        ax.annotate(f'{i}', (point[0], point[1]), 
                   xytext=(5, 5), textcoords='offset points', fontsize=9)
    
    ax.set_title(f'k = {k} кластеров (Инерция: {kmeans.inertia_:.2f})', fontsize=12)
    ax.set_xlabel('Признак 1', fontsize=11)
    ax.set_ylabel('Признак 2', fontsize=11)
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Выводим информацию о кластерах
    print(f"\n{'='*60}")
    print(f"k = {k} кластеров:")
    print(f"  Инерция: {kmeans.inertia_:.2f}")
    print(f"  Центроиды:")
    for i, center in enumerate(centers):
        print(f"    Кластер {i}: {center}")
    print(f"  Метки: {y_pred}")

plt.suptitle('Кластеризация k-means с разным количеством кластеров', fontsize=14)
plt.tight_layout()
plt.show()

# Метод локтя для определения оптимального k
print("\n" + "="*60)
print("Метод локтя для определения оптимального k:")
print("="*60)

inertias = []
K_range = range(1, 8)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)
    print(f"k={k}: инерция={kmeans.inertia_:.2f}")

plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=10)
plt.xlabel('Количество кластеров (k)', fontsize=12)
plt.ylabel('Инерция (сумма квадратов расстояний)', fontsize=12)
plt.title('Метод локтя для определения оптимального k', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(K_range)
plt.tight_layout()
plt.show()

print("\nВывод: По графику метода локтя оптимальное значение k ≈ 2-3")


## Задание 2 - Кластеризация набора данных ирисов Фишера

**Задача:** Выполнить кластеризацию для набора данных ирисов. Поэкспериментировать с количеством кластеров.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Загрузка данных ирисов
df_iris = pd.read_csv('data/iris.csv')

print("Набор данных Iris:")
print(df_iris.head())
print(f"\nРазмер: {df_iris.shape}")
print(f"Классы: {df_iris['variety'].unique()}")
print(f"Распределение классов:\n{df_iris['variety'].value_counts()}")

# Подготовка данных (используем только числовые признаки)
X_iris = df_iris.drop('variety', axis=1).values
y_true = df_iris['variety'].values

# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_iris)

# Экспериментируем с разным количеством кластеров
k_values = [2, 3, 4, 5]

# Визуализация в 2D (используем первые два признака)
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for idx, k in enumerate(k_values):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    ax = axes[idx // 2, idx % 2]
    scatter = ax.scatter(df_iris['sepal.length'], df_iris['sepal.width'],
                        c=clusters, s=100, cmap='viridis', alpha=0.6, 
                        edgecolors='black')
    ax.set_title(f'k = {k} кластеров (Инерция: {kmeans.inertia_:.2f})', fontsize=12)
    ax.set_xlabel('Длина чашелистика', fontsize=11)
    ax.set_ylabel('Ширина чашелистика', fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Кластер')
    
    print(f"\n{'='*60}")
    print(f"k = {k}:")
    print(f"  Инерция: {kmeans.inertia_:.2f}")
    
    # Анализ соответствия кластеров истинным классам
    for cluster_id in range(k):
        mask = clusters == cluster_id
        species_in_cluster = y_true[mask]
        unique, counts = np.unique(species_in_cluster, return_counts=True)
        print(f"  Кластер {cluster_id}: {dict(zip(unique, counts))}")

plt.suptitle('Кластеризация Iris с разным k (sepal.length vs sepal.width)', fontsize=14)
plt.tight_layout()
plt.show()

# Визуализация в другой проекции (petal.length vs petal.width)
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for idx, k in enumerate(k_values):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    ax = axes[idx // 2, idx % 2]
    scatter = ax.scatter(df_iris['petal.length'], df_iris['petal.width'],
                        c=clusters, s=100, cmap='plasma', alpha=0.6, 
                        edgecolors='black')
    ax.set_title(f'k = {k} кластеров', fontsize=12)
    ax.set_xlabel('Длина лепестка', fontsize=11)
    ax.set_ylabel('Ширина лепестка', fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Кластер')

plt.suptitle('Кластеризация Iris с разным k (petal.length vs petal.width)', fontsize=14)
plt.tight_layout()
plt.show()

# Метод локтя
inertias_iris = []
K_range = range(1, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias_iris.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias_iris, 'ro-', linewidth=2, markersize=10)
plt.xlabel('Количество кластеров (k)', fontsize=12)
plt.ylabel('Инерция', fontsize=12)
plt.title('Метод локтя для набора данных Iris', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xticks(K_range)
plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("ВЫВОД:")
print("="*60)
print("Оптимальное количество кластеров k = 3 (соответствует трём видам ирисов)")
print("При k=3 кластеризация хорошо соответствует истинным классам")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

# Создаем набор данных
np.random.seed(42)
X_hier = np.array([[1, 2], [1.5, 1.8], [5, 8], [8, 8], [1, 0.6], [9, 11]])

print("Данные для иерархической кластеризации:")
print(X_hier)
print(f"Размер: {X_hier.shape}")

# Построение дендрограммы
Z = linkage(X_hier, method='ward')

plt.figure(figsize=(12, 6))
dendrogram(Z)
plt.title('Дендрограмма для иерархической кластеризации', fontsize=14)
plt.xlabel('Индексы объектов', fontsize=12)
plt.ylabel('Расстояние', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

# Применяем иерархическую кластеризацию с разным количеством кластеров
cluster_numbers = [2, 3, 4]
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, n_clusters in enumerate(cluster_numbers):
    # Создаем модель
    hc = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    labels = hc.fit_predict(X_hier)
    
    # Визуализация
    ax = axes[idx]
    scatter = ax.scatter(X_hier[:, 0], X_hier[:, 1], c=labels, s=200, 
                        cmap='viridis', alpha=0.6, edgecolors='black', linewidths=2)
    
    # Подписываем точки
    for i, point in enumerate(X_hier):
        ax.annotate(f'{i}', (point[0], point[1]), 
                   xytext=(5, 5), textcoords='offset points', fontsize=11)
    
    ax.set_title(f'{n_clusters} кластера', fontsize=12)
    ax.set_xlabel('X', fontsize=11)
    ax.set_ylabel('Y', fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Кластер')
    
    print(f"\n{n_clusters} кластера: метки = {labels}")

plt.suptitle('Иерархическая кластеризация с разным количеством кластеров', fontsize=14)
plt.tight_layout()
plt.show()


## Задание 3 - Иерархическая кластеризация набора данных Iris

**Задача:** Выполнить иерархическую кластеризацию для набора данных ирисов Фишера, используя любые два признака.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler

# Загрузка данных ирисов
df_iris = pd.read_csv('data/iris.csv')

print("Иерархическая кластеризация набора данных Iris")
print("="*60)
print("Используем признаки: petal.length и petal.width")
print("="*60)

# Выбираем два признака для визуализации
X_2d = df_iris[['petal.length', 'petal.width']].values
y_true = df_iris['variety'].values

# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_2d)

print(f"\nРазмер данных: {X_scaled.shape}")

# Построение дендрограммы
print("\nПостроение дендрограммы...")
Z = linkage(X_scaled, method='ward')

plt.figure(figsize=(15, 6))
dendrogram(Z, truncate_mode='lastp', p=30, show_leaf_counts=True)
plt.title('Дендрограмма для Iris (petal.length vs petal.width)', fontsize=14)
plt.xlabel('Индекс кластера / Количество точек', fontsize=12)
plt.ylabel('Расстояние', fontsize=12)
plt.axhline(y=7, color='r', linestyle='--', linewidth=2, label='Порог для 3 кластеров')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\nПо дендрограмме видно, что оптимальное количество кластеров = 3")

# Применяем иерархическую кластеризацию с разным количеством кластеров
cluster_numbers = [2, 3, 4, 5]
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

for idx, n_clusters in enumerate(cluster_numbers):
    hc = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    labels = hc.fit_predict(X_scaled)
    
    ax = axes[idx // 2, idx % 2]
    scatter = ax.scatter(df_iris['petal.length'], df_iris['petal.width'],
                        c=labels, s=100, cmap='viridis', alpha=0.6, 
                        edgecolors='black', linewidths=1.5)
    
    ax.set_title(f'{n_clusters} кластера', fontsize=13)
    ax.set_xlabel('Длина лепестка (petal.length)', fontsize=11)
    ax.set_ylabel('Ширина лепестка (petal.width)', fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Кластер')
    
    # Анализ соответствия
    print(f"\n{n_clusters} кластера:")
    for cluster_id in range(n_clusters):
        mask = labels == cluster_id
        species_in_cluster = y_true[mask]
        unique, counts = np.unique(species_in_cluster, return_counts=True)
        print(f"  Кластер {cluster_id}: {dict(zip(unique, counts))}")

plt.suptitle('Иерархическая кластеризация Iris с разным количеством кластеров', fontsize=14)
plt.tight_layout()
plt.show()

# Сравнение с истинными классами при n_clusters=3
print("\n" + "="*60)
print("Сравнение с истинными классами (n_clusters=3):")
print("="*60)

hc_final = AgglomerativeClustering(n_clusters=3, linkage='ward')
labels_final = hc_final.fit_predict(X_scaled)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Истинные классы
for species in df_iris['variety'].unique():
    mask = df_iris['variety'] == species
    ax1.scatter(df_iris.loc[mask, 'petal.length'], 
               df_iris.loc[mask, 'petal.width'],
               label=species, s=100, alpha=0.6, edgecolors='black')
ax1.set_xlabel('Длина лепестка', fontsize=11)
ax1.set_ylabel('Ширина лепестка', fontsize=11)
ax1.set_title('Истинные классы', fontsize=13)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Результат кластеризации
scatter = ax2.scatter(df_iris['petal.length'], df_iris['petal.width'],
                     c=labels_final, s=100, cmap='viridis', alpha=0.6, 
                     edgecolors='black')
ax2.set_xlabel('Длина лепестка', fontsize=11)
ax2.set_ylabel('Ширина лепестка', fontsize=11)
ax2.set_title('Результат иерархической кластеризации (3 кластера)', fontsize=13)
plt.colorbar(scatter, ax=ax2, label='Кластер')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nЗадание выполнено!")


## Заключение

Рабочая тетрадь 8 выполнена!

**Что было изучено:**

1. Метод k-средних (k-means)
   - Основы алгоритма
   - Применение на различных данных
   - Метод локтя для выбора оптимального k
2. Иерархическая кластеризация
   - Агломеративная кластеризация
   - Дендрограммы
   - Выбор количества кластеров

**Результаты:**

- K-means успешно применен к синтетическим данным, цифрам и ирисам
- Метод локтя помог определить оптимальное количество кластеров
- Иерархическая кластеризация показала схожие результаты с k-means
- Для набора Iris оптимальное k = 3 (соответствует трём видам)

**Выводы:**

- K-means эффективен для быстрой кластеризации больших данных
- Иерархическая кластеризация дает более гибкие возможности анализа
- Выбор метода зависит от задачи и структуры данных

Все задания выполнены! 🎉
